# Clustering Code Along

We'll be working with a real data set about seeds, from UCI repository: https://archive.ics.uci.edu/ml/datasets/seeds.

The examined group comprised kernels belonging to three different varieties of wheat: Kama, Rosa and Canadian, 70 elements each, randomly selected for 
the experiment. High quality visualization of the internal kernel structure was detected using a soft X-ray technique. It is non-destructive and considerably cheaper than other more sophisticated imaging techniques like scanning microscopy or laser technology. The images were recorded on 13x18 cm X-ray KODAK plates. Studies were conducted using combine harvested wheat grain originating from experimental fields, explored at the Institute of Agrophysics of the Polish Academy of Sciences in Lublin. 

The data set can be used for the tasks of classification and cluster analysis.


Attribute Information:

To construct the data, seven geometric parameters of wheat kernels were measured: 
1. area A, 
2. perimeter P, 
3. compactness C = 4*pi*A/P^2, 
4. length of kernel, 
5. width of kernel, 
6. asymmetry coefficient 
7. length of kernel groove. 
All of these parameters were real-valued continuous.

Let's see if we can cluster them in to 3 groups with K-means!

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [0]:
from pyspark.ml.clustering import KMeansModel

# Loads data.
dataset = spark.sql("SELECT * FROM seeds")

In [0]:
dataset.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|
14.88| 14.57| 0.8811| 5.553999999999999| 3.333| 1.018| 4.956|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|
13.84| 13.94| 0.8955| 5.324|3.3789999999999996| 2.259| 4.805|
16.14| 14.99| 0.9034|5.6579999999999995| 3.562| 1.355| 5.175|
14.38| 14.21| 0.8951| 5.386| 3.312| 2.4619999999999997| 4.956|
14.69| 14.49| 0.8799| 5.563| 3.259| 3.5860000000000003| 5.218999999999999|
14.11| 14.1| 0.8911| 5.42| 3.302| 2.7| 5.0|
16.63| 15.46| 0.8747| 6.053| 3.465| 2.04| 5.877000000000001|
16.44| 15.25| 0.888|5.8839999999999995| 3.505| 1.969|5.5329999999999995|
15.26| 14.85| 0.8696|5.7139999999999995| 3.242| 4.543| 5.314|
14.03| 14.16| 0.8796| 5.438| 3.201| 1.7169999999999999| 5.001|
13.89| 14.02| 0.888| 5.439| 3.199| 3.986| 4.738|
13.78| 14.06| 0.8759| 5.479| 3.156| 3.136| 4.872|
13.74| 14.05| 0.8744| 5.482| 3.114| 2.932| 4.825|
14.59| 14.28| 0.8993| 5.351| 3.333| 4.185| 4.781000000000001|
13.99| 13.83| 0.9183| 5.119| 3.383| 5.234| 4.781000000000001|
15.69| 14.75| 0.9058| 5.527| 3.514| 1.599| 5.046|
 14.7| 14.21| 0.9153| 5.205| 3.466| 1.767| 4.649|
12.72| 13.57| 0.8686| 5.226| 3.049| 4.102| 4.914|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
only showing top 20 rows

In [0]:
dataset.head()

Out[26]: Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)

In [0]:
dataset.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
summary| area| perimeter| compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
 count| 210| 210| 210| 210| 210| 210| 210|
 mean|14.847523809523816|14.559285714285718| 0.8709985714285714| 5.628533333333335| 3.258604761904762| 3.7001999999999997| 5.408071428571429|
 stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867| 1.5035589702547392|0.49148049910240543|
 min| 10.59| 12.41| 0.8081| 4.899| 2.63| 0.765| 4.519|
 max| 21.18| 17.25| 0.9183| 6.675| 4.033| 8.456| 6.55|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+

## Format the Data

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
dataset.columns

Out[29]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [0]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [0]:
final_data = vec_assembler.transform(dataset)

## Scale the Data
It is a good idea to scale our data to deal with the curse of dimensionality: https://en.wikipedia.org/wiki/Curse_of_dimensionality

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [0]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [0]:
# Normalize each feature to have unit standard deviation.
final_data = scalerModel.transform(final_data)

In [0]:
final_data.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove| features| scaledFeatures|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|[15.26,14.84,0.87...|[5.24452795332028...|
14.88| 14.57| 0.8811| 5.553999999999999| 3.333| 1.018| 4.956|[14.88,14.57,0.88...|[5.11393027165175...|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|[14.29,14.09,0.90...|[4.91116018695588...|
13.84| 13.94| 0.8955| 5.324|3.3789999999999996| 2.259| 4.805|[13.84,13.94,0.89...|[4.75650503761158...|
16.14| 14.99| 0.9034|5.6579999999999995| 3.562| 1.355| 5.175|[16.14,14.99,0.90...|[5.54696468981581...|
14.38| 14.21| 0.8951| 5.386| 3.312| 2.4619999999999997| 4.956|[14.38,14.21,0.89...|[4.94209121682475...|
14.69| 14.49| 0.8799| 5.563| 3.259| 3.5860000000000003| 5.218999999999999|[14.69,14.49,0.87...|[5.04863143081749...|
14.11| 14.1| 0.8911| 5.42| 3.302| 2.7| 5.0|[14.11,14.1,0.891...|[4.84929812721816...|
16.63| 15.46| 0.8747| 6.053| 3.465| 2.04| 5.877000000000001|[16.63,15.46,0.87...|[5.71536696354628...|
16.44| 15.25| 0.888|5.8839999999999995| 3.505| 1.969|5.5329999999999995|[16.44,15.25,0.88...|[5.65006812271202...|
15.26| 14.85| 0.8696|5.7139999999999995| 3.242| 4.543| 5.314|[15.26,14.85,0.86...|[5.24452795332028...|
14.03| 14.16| 0.8796| 5.438| 3.201| 1.7169999999999999| 5.001|[14.03,14.16,0.87...|[4.82180387844584...|
13.89| 14.02| 0.888| 5.439| 3.199| 3.986| 4.738|[13.89,14.02,0.88...|[4.77368894309428...|
13.78| 14.06| 0.8759| 5.479| 3.156| 3.136| 4.872|[13.78,14.06,0.87...|[4.73588435103234...|
13.74| 14.05| 0.8744| 5.482| 3.114| 2.932| 4.825|[13.74,14.05,0.87...|[4.72213722664617...|
14.59| 14.28| 0.8993| 5.351| 3.333| 4.185| 4.781000000000001|[14.59,14.28,0.89...|[5.01426361985209...|
13.99| 13.83| 0.9183| 5.119| 3.383| 5.234| 4.781000000000001|[13.99,13.83,0.91...|[4.80805675405968...|
15.69| 14.75| 0.9058| 5.527| 3.514| 1.599| 5.046|[15.69,14.75,0.90...|[5.39230954047151...|
 14.7| 14.21| 0.9153| 5.205| 3.466| 1.767| 4.649|[14.7,14.21,0.915...|[5.05206821191403...|
12.72| 13.57| 0.8686| 5.226| 3.049| 4.102| 4.914|[12.72,13.57,0.86...|[4.37158555479908...|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+
only showing top 20 rows

## Train the Model and Evaluate

In [0]:
# Trains a k-means model.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [0]:
predictions = model.transform(final_data)

In [0]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))


Silhouette with squared euclidean distance = 0.616267393520126

In [0]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 6.31670546 12.37109759 37.39491396 13.91155062 9.748067
 2.39849968 12.2661748 ]
[ 4.87257659 10.88120146 37.27692543 12.3410157 8.55443412
 1.81649011 10.32998598]
[ 4.06105916 10.13979506 35.80536984 11.82133095 7.50395937
 3.27184732 10.42126018]

In [0]:
model.transform(final_data).select('prediction').show()

+----------+
prediction|
+----------+
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 0|
 0|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 2|
+----------+
only showing top 20 rows